In [1]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import pymysql  # ✅ Correct connector

# API details
api_key = "9vwTpZnV8T3jHXHKfpAU116yzoSYHMJJ8iimvq7w"
url = f"https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key={api_key}"

# Fetch data from API
data = requests.get(url).json()

# Connect to MySQL using PyMySQL
conn = pymysql.connect(
    host="127.0.0.1",
    user="root",
    password="Laptop@321",
    database="tennis"
)
cursor = conn.cursor()

# Insert data into MySQL tables
for comp in data.get("competitions", []):
    cat = comp["category"]

    # Insert or update into Categories table
    cursor.execute("""
        INSERT INTO Categories (category_id, category_name) 
        VALUES (%s, %s) 
        ON DUPLICATE KEY UPDATE category_name = VALUES(category_name)
    """, (cat["id"], cat["name"]))

    # Insert or update into Competitions table
    cursor.execute("""
        INSERT INTO Competitions (competition_id, competition_name, parent_id, type, gender, category_id) 
        VALUES (%s, %s, %s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
            competition_name = VALUES(competition_name), 
            parent_id = VALUES(parent_id), 
            type = VALUES(type), 
            gender = VALUES(gender), 
            category_id = VALUES(category_id)
    """, (
        comp["id"],
        comp["name"],
        comp.get("parent_id"),
        comp.get("type"),
        comp.get("gender"),
        cat["id"]
    ))

# Finalize
conn.commit()
cursor.close()
conn.close()
